In [1]:
import numpy as np
import pprint

In [2]:
class PSimpleExp():
    def __init__(self,n_outcomes,alphabet=None,prior_prob=None,n_trials=None):
        # Type control for alphabet: list or numpy array
        if alphabet is not None:
            if type(alphabet) == list:
                self.alphabet = alphabet
            elif isinstance(alphabet,np.ndarray):
                self.alphabet = list(alphabet)
            else:
                raise TypeError('Alphabet must be a list or numpy array')
            # If alphabet is not null, it overrides n_outcomes
            self.n_outcomes = len(alphabet)
            # Alphabet items must be string
            self.alphabet = [str(c) for c in self.alphabet]
        else:
            # If alphabet is null, n_outcomes is given by the input variable
            # and alphabet is a list of characters given by the int sequence
            if (type(n_outcomes) == int) and (n_outcomes>1):
                self.n_outcomes = n_outcomes
                self.alphabet = [str(i+1) for i in range(self.n_outcomes)]
            else:
                raise TypeError('n_outcomes must be a positive integer greater than 1')
                
        # Type control for prior_prob: list or numpy array
        if prior_prob is not None:
            if type(prior_prob) == list:
                self.prior_prob = np.array(prior_prob,dtype='float')
            elif isinstance(alphabet,np.ndarray):
                self.prior_prob = prior_prob
            else:
                raise TypeError('prior_prob must be a list or numpy array')
            # prior_prob is normalized
            self.prior_prob /= np.sum(self.prior_prob)
        # If prior_prob is null, a uniform distribution is set
        else:
            self.prior_prob = np.ones(self.n_outcomes)/self.n_outcomes
            
        # If prior_prob and alphabet have different lengths, alphabet and n_outcomes are accordingly adapted
        if len(self.prior_prob) != len(self.alphabet):
            atemp = self.alphabet.copy()
            if len(self.prior_prob) < len(self.alphabet):
                self.alphabet = None
                self.alphabet = atemp[0:len(self.prior_prob)]
            else:
                alphabet_set = set(atemp)
                ctemp = 1
                while len(self.alphabet) < len(self.prior_prob):
                    ctempstr = str(ctemp)
                    if ctempstr not in alphabet_set:
                        self.alphabet.append(ctempstr)
                    ctemp += 1
            self.n_outcomes = len(self.prior_prob)
        
        # Setting n_trials
        if n_trials is not None:
            if (type(n_trials) is not int) or (n_trials <= 0):
                raise TypeError('n_trials must be a positive integer')
            self.n_trials = n_trials
        else:
            self.n_trials = self.n_outcomes * 10000
            
        # exp will contain the samples of the experiment -> make_exp
        self.exp = np.zeros(self.n_trials)
        # post_prob will contain the estimated probabilities from the experiment -> make_exp
        self.post_prob = np.zeros(self.n_outcomes)
        
        self.prob_dict = {}
        for k in range(self.n_outcomes):
            self.prob_dict[self.alphabet[k]] = [self.prior_prob[k],None]
    
    def make_exp(self):
        unif = np.random.uniform(size=self.n_trials)
        csp = np.cumsum(self.prior_prob)
        unif = np.random.uniform(size=e.n_trials)
        csp = np.cumsum(self.prior_prob)
        p0=0.0
        for k in range(self.n_outcomes):
            self.post_prob[k] = np.sum(csp[k] > unif)/self.n_trials - p0
            p0 += self.post_prob[k]
            self.prob_dict[self.alphabet[k]][1] = self.post_prob[k]
    
    def event_probability(self,ev):
        p0 = 0.0
        p1 = 0.0
        for k in ev.abc:
            if k not in self.alphabet:
                raise ValueError('Event abc must be a subset of the experiment alphabet')
            p0 += self.prob_dict[k][0]
            p1 += self.prob_dict[k][1]
        return p0, p1

In [3]:
e = PSimpleExp(6,prior_prob=[1,1,2,1,1,1],n_trials=10000)
print(e.n_outcomes)
print(e.alphabet)
print(e.prior_prob,np.sum(e.prior_prob))
print(e.n_trials)
pprint.pprint(e.prob_dict)

6
['1', '2', '3', '4', '5', '6']
[ 0.14285714  0.14285714  0.28571429  0.14285714  0.14285714  0.14285714] 1.0
10000
{'1': [0.14285714285714285, None],
 '2': [0.14285714285714285, None],
 '3': [0.2857142857142857, None],
 '4': [0.14285714285714285, None],
 '5': [0.14285714285714285, None],
 '6': [0.14285714285714285, None]}


In [4]:
e.make_exp()
pprint.pprint(e.prob_dict)

{'1': [0.14285714285714285, 0.1464],
 '2': [0.14285714285714285, 0.1419],
 '3': [0.2857142857142857, 0.28320000000000001],
 '4': [0.14285714285714285, 0.1462],
 '5': [0.14285714285714285, 0.13639999999999997],
 '6': [0.14285714285714285, 0.14590000000000003]}


In [15]:
class Event():
    def __init__(self,abc,alphabet):
        if (type(alphabet) != list) and (not isinstance(alphabet,np.ndarray)):
            raise TypeError('Alphabet must be a list or numpy array of strings')
        else:
            self.alphabet = [str(c) for c in alphabet]
        if (type(abc) != list) and (not isinstance(abc,np.ndarray)):
            raise TypeError('abc must be a list or numpy array of strings')
        else:
            self.abc = [str(c) for c in abc]
        for c in self.abc:
            if c not in self.alphabet:
                raise ValueError('abc must be a subset of alphabet')
        
    def __neg__(self):
        abctemp = []
        for k in self.alphabet:
            if k not in self.abc:
                abctemp.append(k)
        return Event(abctemp,self.alphabet)
    
    def __add__(self,right):
        if type(right) != Event:
            raise TypeError('All the operands must be Event objects')
        abctemp = list(set(self.abc + right.abc)) # Lists concatenation
        abctemp.sort()
        return Event(abctemp,self.alphabet)
    
    def __mul__(self,right):
        if type(right) != Event:
            raise TypeError('All the operands must be Event objects')
        abctemp = []
        for k in self.abc:
            if k in right.abc:
                abctemp.append(k)
        return Event(abctemp,self.alphabet)
    
    def __repr__(self):
        return(str(self.abc))
    def __str__(self):
        return(str(self.abc))

In [17]:
type(e.alphabet) != list
event1 = Event(['1','2'],e.alphabet)
print('event1: ' + str(event1))
print('event1 reference alphabet: ' + str(event1.alphabet))
print('NOT event1: ' + str(-event1))

event2 = Event(['1','3'],e.alphabet)
print('event2: ' + str(event2))
#print(event2.alphabet)
print('event1 OR event2: ' + str(event1+event2))
print('event1 AND event2: ' + str(event1*event2))

emptyevent = Event([],e.alphabet)
print('NOT emptyevent: ' + str(-emptyevent))

print('event2 AND emptyevent: ' + str(event2*emptyevent))

event1: ['1', '2']
event1 reference alphabet: ['1', '2', '3', '4', '5', '6']
NOT event1: ['3', '4', '5', '6']
event2: ['1', '3']
event1 OR event2: ['1', '2', '3']
event1 AND event2: ['1']
NOT emptyevent: ['1', '2', '3', '4', '5', '6']
event2 AND emptyevent: []


In [13]:
e.event_probability(event1*event2)

(0.14285714285714285, 0.1464)

In [8]:
e.event_probability(event1+event2)

(0.5714285714285714, 0.57150000000000001)

In [9]:
np.sum(e.prior_prob[:3])

0.5714285714285714

In [10]:
np.sum(e.post_prob[:3])

0.57150000000000001

In [72]:
class PComposedExp(PSimpleExp):
    def __init__(self,explist,n_trials=None):
        PSimpleExp.__init__(self,2)
        if type(explist) != list:
            raise TypeError('Argument must be a list of experiments')
        for e in explist:
            if (type(e) != PSimpleExp) and (type(e) != PComposedExp):
                raise TypeError('Argument must be a list of experiments')
        
        # Alphabet is the Cartesian Product of the single experiments
        self.n_outcomes = 1
        for exp in explist:
            self.n_outcomes *= exp.n_outcomes
        self.alphabet = []
        for exp in range(self.n_outcomes):
            self.alphabet.append([])
        
        # Cartesian product
        temp_outc = 1
        for e in explist:
            temp_outc *= e.n_outcomes # 2, 6, 24
            n_outc = int(self.n_outcomes/temp_outc) # 12, 4, 1
            n_rep = int(self.n_outcomes/(n_outc*e.n_outcomes)) # 1, 2, 6
            indc = 0
            for j in range(n_rep):
                for s in e.alphabet:
                    for k in range(n_outc):
                        self.alphabet[indc].append(s)
                        indc +=1
                        
        # Setting prior_prob
        self.prior_prob = np.ones(self.n_outcomes)
        for a in self.alphabet:
            for p in range(len(a)):
                

In [73]:
exp1 = PSimpleExp(2,alphabet=['M','F'])
exp2 = PSimpleExp(3,alphabet=['p','q','r'])
exp3 = PSimpleExp(4)
compexp = PComposedExp([exp1,exp2,exp3])
compexp.alphabet

[['M', 'p', '1'],
 ['M', 'p', '2'],
 ['M', 'p', '3'],
 ['M', 'p', '4'],
 ['M', 'q', '1'],
 ['M', 'q', '2'],
 ['M', 'q', '3'],
 ['M', 'q', '4'],
 ['M', 'r', '1'],
 ['M', 'r', '2'],
 ['M', 'r', '3'],
 ['M', 'r', '4'],
 ['F', 'p', '1'],
 ['F', 'p', '2'],
 ['F', 'p', '3'],
 ['F', 'p', '4'],
 ['F', 'q', '1'],
 ['F', 'q', '2'],
 ['F', 'q', '3'],
 ['F', 'q', '4'],
 ['F', 'r', '1'],
 ['F', 'r', '2'],
 ['F', 'r', '3'],
 ['F', 'r', '4']]